In [0]:
CREATE TABLE pricing_analytics.gold.reporting_dim_product_gold_SCDTYPE2 (
  PRODUCTGROUP_NAME STRING,
  PRODUCT_NAME STRING,
  PRODUCT_ID BIGINT,
  start_date TIMESTAMP,
  end_date TIMESTAMP,
  lakehouse_inserted_date TIMESTAMP,
  lakehouse_updated_date TIMESTAMP
  )
USING delta

In [0]:
update pricing_analytics.silver.daily_pricing_silver
set productgroup_name = "Vegies",
lakehouse_updated_date = current_timestamp()
where product_name= "Onion"

In [0]:
create or replace table pricing_analytics.silver.reporting_dim_product_stage_1 as 
select distinct  PRODUCTGROUP_NAME, PRODUCT_NAME 
from pricing_analytics.silver.daily_pricing_silver
where lakehouse_updated_date >
(select nvl(max(PROCESSED_TABLE_DATE_TIME), "1900-01-01") from pricing_analytics.processrunlogs.deltalakehouse_process_runs
where PROCESS_NAME="reportingDimensionTableLoadSCDType2" and  PROCESS_STATUS= "Completed")

In [0]:
select * from pricing_analytics.silver.reporting_dim_product_stage_1

In [0]:
create or replace table pricing_analytics.silver.reporting_dim_product_stage_2 as 
select 
silverDim.PRODUCTGROUP_NAME,
silverDim.PRODUCT_NAME,
goldDim.PRODUCT_NAME as GOLD_PRODUCT_NAME,
row_number() over (order by silverDim.PRODUCT_NAME, silverDim.PRODUCTGROUP_NAME)  as PRODUCT_ID,
current_timestamp() as start_date,
CAST(NULL AS TIMESTAMP) AS end_date,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date

from pricing_analytics.silver.reporting_dim_product_stage_1 silverDim
left outer join pricing_analytics.gold.reporting_dim_product_gold_scdtype2 goldDim
on silverDim.PRODUCT_NAME = goldDim.PRODUCT_NAME and isnull(goldDim.end_date)
where isnull(goldDim.PRODUCT_NAME) or silverdim.PRODUCTGROUP_NAME != goldDim.PRODUCTGROUP_NAME

In [0]:
select * from pricing_analytics.silver.reporting_dim_product_stage_2

In [0]:
create or replace table  pricing_analytics.silver.reporting_dim_product_stage_3 as 
select silverDim.PRODUCT_NAME,
silverDim.PRODUCTGROUP_NAME,
(silverDim.PRODUCT_ID + PreviousMax)  as PRODUCT_ID,
start_date,
end_date,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date

from pricing_analytics.silver.reporting_dim_product_stage_2 as silverDim
cross join (select nvl(max(PRODUCT_ID),0) as PreviousMax  from  pricing_analytics.gold.reporting_dim_product_gold_scdtype2) as goldDim

In [0]:
select * from pricing_analytics.silver.reporting_dim_product_stage_3

In [0]:
Merge into pricing_analytics.gold.reporting_dim_product_gold_scdtype2 as goldDim 
using  pricing_analytics.silver.reporting_dim_product_stage_3 as silverDim 
on goldDim.PRODUCT_NAME = silverDim.PRODUCT_NAME and isnull(goldDim.end_date)
when matched then 
update set goldDim.end_date = current_timestamp(),
goldDim.lakehouse_updated_date = current_timestamp();


In [0]:
insert into pricing_analytics.gold.reporting_dim_product_gold_scdtype2
select PRODUCTGROUP_NAME, PRODUCT_NAME, PRODUCT_ID,  start_date, end_date, lakehouse_inserted_date, lakehouse_updated_date from pricing_analytics.silver.reporting_dim_product_stage_3;

In [0]:
select * from pricing_analytics.gold.reporting_dim_product_gold_scdtype2 where PRODUCT_NAME ='Onion'

In [0]:
insert into pricing_analytics.processrunlogs.deltalakehouse_process_runs(PROCESS_NAME,PROCESS_STATUS,PROCESSED_TABLE_DATE_TIME)
select "reportingDimensionTableLoadSCDType2", "Completed", max(lakehouse_updated_date) from pricing_analytics.silver.daily_pricing_silver